# Tutorial for Slide-seqV2 data

## 1. Convert data

The data used in this notebook comes from `Puck_200903_13` from [High Resolution Slide-seqV2 Spatial Transcriptomics Enables Discovery of Disease-Specific Cell Neighborhoods and Pathways](https://doi.org/10.1016/j.isci.2022.104097) which can be downloaded from https://cellxgene.cziscience.com/collections/8e880741-bf9a-4c8e-9227-934204631d2a.

This notebook assumes the data is located at `./data/marshall_2022_iscience.h5ad`.

In [4]:
import scanpy as sc
import numpy as np
import scipy.cluster
from scipy import sparse

from anndata import read_h5ad
from os.path import join

from vitessce.data_utils import (
    to_diamond,
    to_uint8,
    optimize_adata,
)

In [5]:
slide_seq_h5ad = join("data", "marshall_2022_iscience.h5ad")
slide_seq_zarr = join("data", "marshall_2022_iscience.h5ad.zarr")

In [7]:
# Read the original H5AD file from cellxgene
adata = read_h5ad(slide_seq_h5ad)

# Use ScanPy to preprocess the data,
# perform quality control, filtering,
# normalization, scaling, and
# identification of highly variable genes.
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

adata.var['mt'] = adata.var['feature_name'].str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

sc.pp.normalize_total(adata, target_sum=1e4, inplace=True)
sc.pp.log1p(adata)

sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

# Store the highly variable genes in a smaller cell-by-gene expression matrix.
adata_hvg = adata[:, adata.var['highly_variable']].copy()
sc.pp.regress_out(adata_hvg, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata_hvg, max_value=3)

adata.obsm['X_hvg'] = adata_hvg.X
adata.obsm['X_hvg_uint8'] = to_uint8(adata_hvg.X, norm_along="var")

# Generate pseudo-segmentations because the data is spatially-resolved but
# does not have cell segmentations.
num_cells = adata.obs.shape[0]
adata.obsm['X_spatial'] = adata.obsm['X_spatial']
adata.obsm['X_segmentations'] = np.zeros((num_cells, 4, 2))
radius = 10
for i in range(num_cells):
    adata.obsm['X_segmentations'][i, :, :] = to_diamond(adata.obsm['X_spatial'][i, 0], adata.obsm['X_spatial'][i, 1], radius)


# Use the optimize_adata function to select only the columns/keys that
# are necessary for visualization.
adata = optimize_adata(
    adata,
    obs_cols=["cell_type"],
    var_cols=["feature_name", "highly_variable"],
    obsm_keys=["X_hvg", "X_hvg_uint8", "X_umap", "X_spatial", "X_segmentations"],
    layer_keys=[],
)

# Save the processed AnnData object to Zarr format.
adata.write_zarr(slide_seq_zarr, chunks=[adata.shape[0], 10])

/Users/mkeller/anaconda3/envs/vitessce-tutorial-env/lib/python3.8/site-packages/zarr/creation.py:241: UserWarning: ignoring keyword argument 'maxshape'
  warn('ignoring keyword argument %r' % k)


## 2. Configure visualization

In [12]:
from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    FileType as ft,
    AnnDataWrapper,
    OmeTiffWrapper,
)

### Instantiate a `VitessceConfig` object

We begin the configuration process by creating an object using the `VitessceConfig` constructor. This takes three parameters:
- `schema_version`: the schema version that the configuration should conform to. Valid values can be found at http://vitessce.io/docs/view-config-json/#version. The current most recent version `1.0.15` (as of 1/16/2023) is used in this tutorial.
- `name`: a name for the configuration.
- `description` (optional): a brief description of the configuration.


In [13]:
vc = VitessceConfig(schema_version="1.0.15", name='Slide-seqV2', description='Kidney')

### Add data

To add data to the configuration, we first run [add_dataset](https://vitessce.github.io/vitessce-python/api_config.html#vitessce.config.VitessceConfig.add_dataset) which takes the dataset `name` as a parameter.

This returns a new `VitessceConfigDataset` instance. We can add objects which represent local data such as AnnData stores by running [add_object](https://vitessce.github.io/vitessce-python/api_config.html#vitessce.config.VitessceConfigDataset.add_object) on the dataset instance. To enable multiple `add_object` calls to be chained together, the `add_object` function also returns the `VitessceConfigDataset` instance.

We will store the `VitessceConfigDataset` instance in a variable (`dataset`) to use later when configuring views.

__Note__: the functions used in the following cells `.add_dataset`, `.add_object`, and `.add_view` have side effects (i.e., they modify the `vc` object). Running these cells more than once on the same `vc` instance may result in an unexpected configuration, so be sure to run the cells in order starting from the initial `vc = VitessceConfig(...)` cell.

In [14]:
dataset = vc.add_dataset(name='Puck_200903_13').add_object(AnnDataWrapper(
    # This file path points to the AnnData object written to Zarr format.
    adata_path=slide_seq_zarr,
    # Paths to dimensionality reduction(s) within the AnnData object.
    obs_embedding_paths=["obsm/X_umap"],
    obs_embedding_names=["UMAP"],
    # Path to array containing spatial coordinates.
    obs_locations_path="obsm/X_spatial",
    # Path to array containing polygon cell segmentations (or the pseudo-segmentations in this case).
    obs_segmentations_path="obsm/X_segmentations",
    # Path to column in adata containing cell type annotations.
    obs_set_paths=["obs/cell_type"],
    obs_set_names=["Bead type"],
    # Path to cell-by-gene matrix.
    obs_feature_matrix_path="obsm/X_hvg",
    # If cell-by-gene matrix is a filtered version of the entire adata.X,
    # then we add a path to a column containing boolean flags that was used to filter the matrix.
    feature_filter_path="var/highly_variable",
    # If the observations do not represent cells, we specify a different observation type using `obsType` here.
    coordination_values={
        "obsType": "bead"
    }
))

### Add views

Next, we configure the visualization and controller views of interest.

In [15]:
obs_sets = vc.add_view(vt.OBS_SETS, dataset=dataset)
obs_set_sizes = vc.add_view(vt.OBS_SET_SIZES, dataset=dataset)
scatterplot = vc.add_view(vt.SCATTERPLOT, dataset=dataset, mapping="UMAP")
spatial = vc.add_view(vt.SPATIAL, dataset=dataset)
layer_controller = vc.add_view(vt.LAYER_CONTROLLER, dataset=dataset)
gene_list = vc.add_view(vt.FEATURE_LIST, dataset=dataset)

### Coordinate views

Views can be linked on different properties by using the [link_views](https://vitessce.github.io/vitessce-python/api_config.html#vitessce.config.VitessceConfig.link_views) function.

In [16]:
# We need to specify a new observation type for all views, so that these views
# know to load data from the AnnData object that has "obsType": "bead".
vc.link_views([obs_sets, obs_set_sizes, scatterplot, spatial, layer_controller, gene_list], [ct.OBS_TYPE], ["bead"])

In [17]:
# We need to specify that we want to view the pseudo-segmentations in the spatial view,
# by providing an initial value for ct.SPATIAL_SEGMENTATION_LAYER here.
vc.link_views([spatial, layer_controller], [ct.SPATIAL_SEGMENTATION_LAYER], [{ "opacity": 1, "radius": 50, "visible": True, "stroked": False }])

In [18]:
# We define a layout for the plots
vc.layout(
    (scatterplot | (gene_list | obs_sets))
    / (spatial | (layer_controller | obs_set_sizes))
);

### Render the widget into the notebook

To render the interactive visualization into the notebook, we run the [widget](https://vitessce.github.io/vitessce-python/api_config.html#vitessce.config.VitessceConfig.widget) function. 

In [19]:
vw = vc.widget()
vw

VitessceWidget(config={'version': '1.0.15', 'name': 'Slide-seqV2', 'description': 'Kidney', 'datasets': [{'uid…